In [1]:
# install requests if not already installed 
#!pip install requests

# first step: install and create a folder
# as account identifier to secure your snowflake login details(.env)

#!pip install python-dotenv
#!pip install snowflake-sqlalchemy
#!pip install snowflake-connector-python


In [2]:
#!pip install openpyxl

In [3]:
# importing librabries
import requests
import pandas as pd
from datetime import datetime

# load to Snowflake DB
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from snowflake.sqlalchemy import URL


C:\Users\back2\AppData\Roaming\Python\Python39\site-packages\snowflake\connector\options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [4]:
# making a request( api call)
api_key = '6b9effe684bfcde6864f4da66277cede'
lon = 5.47631
lat = 7.025853
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}'

response = requests.get(url)

In [5]:
# check your status code to be sure you got the response you expected 
# if output is 200 then you good to go
response.status_code

200

In [6]:
# view response
data = response.json()
data

{'coord': {'lon': 5.4763, 'lat': 7.0259},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 297.35,
  'feels_like': 298.18,
  'temp_min': 297.35,
  'temp_max': 297.35,
  'pressure': 1012,
  'humidity': 90,
  'sea_level': 1012,
  'grnd_level': 991},
 'visibility': 10000,
 'wind': {'speed': 1.54, 'deg': 244, 'gust': 5.61},
 'clouds': {'all': 58},
 'dt': 1747861878,
 'sys': {'country': 'NG', 'sunrise': 1747804852, 'sunset': 1747849729},
 'timezone': 3600,
 'id': 2325314,
 'name': 'Owo',
 'cod': 200}

In [7]:
# ways of extracting different data from  lists and dictionaries
#  in the case of wearther the dictionary is in the list.to get an element in a list you
#  subset for the first element in that list which is 0

#data['weather']

#data['weather'][0]

#weather= data['weather'][0]['main']

#data['main']

#temp = data['main']['temp']
#temp

#pressure = data['main']['pressure']
#pressure

In [8]:
# extracting relevant data informations or demands

location= data['name']
weather = data['weather'][0]['main']
description = data['weather'][0]['description']
temp = data['main']['temp']
pressure = data['main']['pressure']
humidity = data['main']['humidity']
speed = data['wind']['speed']
Date = data['dt']
time = data['timezone']

### next step create DataFrame with pandas

In [9]:
### next step create DataFrame with pandas
# At this stage you have extracted from Data_warehouse and you are building a data_pipeline( ETL)
df = pd.DataFrame([{'location': location,
                    'weather': weather,
                    'description': description,
                    'temperature': temp,
                    'pressure': pressure,
                    'humidity': humidity,
                    'speed': speed,
                    'dt': Date,
                    'timezone': time}])
df

,location,weather,description,temperature,pressure,humidity,speed,dt,timezone
0,Owo,Clouds,broken clouds,297.35,1012,90,1.54,1747861878,3600


In [10]:
# step 1 input the excel file into the pipeline
all_locations = pd.read_csv(r'API_WORK_FILES_cities.csv')
all_locations.head()

,city,latitude,longitude
0,Abeokuta,7.145244,3.327695
1,Port Harcourt,4.824167,7.033611
2,Maiduguri,11.833333,13.150000
3,Kano,12.000000,8.516667
4,Katsina,12.985531,7.617144


In [11]:
#step 2:create an empty list that will retain all the relevant info or colunms that is already created.
locations = []
weathers = []
descriptions = []
temps = []
pressures = []
humidities = []
speeds = []
Dates = []
times = []

# step 3 : loop through the excel datatframe and iterate to get the data you want.

for index, row in all_locations.iterrows():
        
        api_key = '6b9effe684bfcde6864f4da66277cede'
        lon = row['longitude']
        lat = row['latitude']
        
        url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}'

        response = requests.get(url)
        data = response.json()
         
        location= data['name']
        locations.append(location)

        weather = data['weather'][0]['main']
        weathers.append(weather)

        description = data['weather'][0]['description']
        descriptions.append(description)

        temp = data['main']['temp']
        temps.append(temp)

        pressure = data['main']['pressure']
        pressures.append(pressure)

        humidity = data['main']['humidity']
        humidities.append(humidity)

        speed = data['wind']['speed']
        speeds.append(speed)

        Date = data['dt']
        Dates.append(Date)

        time = data['timezone']
        times.append(time)

#Creating my DataFrame
df = pd.DataFrame({'location': locations,
                    'weather': weathers,
                    'description': descriptions,
                    'temperature': temps,
                    'pressure': pressures,
                    'humidity': humidities,
                    'speed': speeds,
                    'dt': Dates,
                    'timezone': times})
df



,location,weather,description,temperature,pressure,humidity,speed,dt,timezone
0,Abeokuta,Clouds,scattered clouds,299.07,1012,85,3.41,1747861878,3600
1,Orogbum,Clouds,overcast clouds,299.28,1012,85,1.98,1747861878,3600
2,Maiduguri,Clear,clear sky,304.84,1009,33,2.98,1747861878,3600
3,Kano,Clear,clear sky,307.21,1009,18,2.73,1747861878,3600
4,Katsina,Clear,clear sky,306.10,1009,14,3.92,1747861878,3600
5,Nnewi,Clouds,overcast clouds,299.02,1012,82,3.50,1747861878,3600
6,Agbor,Clouds,broken clouds,297.66,1012,94,2.26,1747861878,3600
7,Ikeja,Clouds,scattered clouds,300.42,1012,80,4.48,1747861879,3600
8,Delta,Clouds,broken clouds,297.70,1012,96,2.24,1747861879,3600
9,Akure,Clouds,broken clouds,296.19,1013,95,1.38,1747861879,3600


### Transformation

In [12]:
# first of all make a copy of your dataframe to avoid tampering with the originally created one
df2 = df.copy()

In [13]:
# first assignment can be done this way 
#def temp_conv(x):
    #new_temp = x - 273.15
    #return new_temp

In [14]:
# First assisgnment can be simply done this way
# convert temperature reading kelvin to celcius
# this function was only applied to a colunm
df2['temperature'].apply(lambda x: x - 273.15) 

0     25.92
1     26.13
2     31.69
3     34.06
4     32.95
5     25.87
6     24.51
7     27.27
8     24.55
9     23.04
10    24.47
11    24.84
12    25.50
13    33.37
14    25.75
15    30.30
16    24.14
17    27.21
18    27.90
19    35.80
20    24.66
21    28.64
22    24.64
23    23.15
24    24.81
25    24.16
26    25.97
27    27.00
28    25.09
29    29.23
30    30.18
31    23.30
32    29.80
33    24.88
34    27.61
35    26.54
36    24.67
37    35.62
38    30.05
39    27.10
40    34.34
41    25.07
42    27.50
43    28.04
44    26.64
45    24.33
Name: temperature, dtype: float64

In [15]:
# second assignment 
#convert date and timezone from seconds to readable format(int timezone you add or subtract)
from datetime import datetime
def adjust_date(row):
    unix_datetime = row['dt'] + row['timezone']

    #conversion
    actual_datetime = datetime.fromtimestamp(unix_datetime)
    return actual_datetime
    
  

In [16]:
# this function is being applied to the colunm and rows
# NB: axis zero(0) is rows while axis one(1) is columns
df2.apply(adjust_date, axis=1)

0    2025-05-21 23:11:18
1    2025-05-21 23:11:18
2    2025-05-21 23:11:18
3    2025-05-21 23:11:18
4    2025-05-21 23:11:18
5    2025-05-21 23:11:18
6    2025-05-21 23:11:18
7    2025-05-21 23:11:19
8    2025-05-21 23:11:19
9    2025-05-21 23:11:19
10   2025-05-21 23:11:19
11   2025-05-21 23:11:19
12   2025-05-21 23:11:19
13   2025-05-21 23:11:19
14   2025-05-21 23:11:19
15   2025-05-21 23:11:19
16   2025-05-21 23:11:20
17   2025-05-21 23:11:20
18   2025-05-21 23:11:20
19   2025-05-21 23:11:20
20   2025-05-21 23:11:20
21   2025-05-21 23:11:20
22   2025-05-21 23:11:20
23   2025-05-21 23:11:20
24   2025-05-21 23:11:20
25   2025-05-21 23:11:20
26   2025-05-21 23:11:21
27   2025-05-21 23:11:21
28   2025-05-21 23:11:21
29   2025-05-21 23:11:21
30   2025-05-21 23:11:21
31   2025-05-21 23:11:21
32   2025-05-21 23:11:21
33   2025-05-21 23:11:21
34   2025-05-21 23:11:21
35   2025-05-21 23:09:35
36   2025-05-21 23:11:21
37   2025-05-21 23:11:22
38   2025-05-21 23:11:22
39   2025-05-21 23:11:22


In [17]:
# create a new colunm called reading date
df2['Reading_date'] = df2.apply(adjust_date, axis=1)



In [18]:
#remove or drop dt and timezone to make table look standard
df2.drop(['dt', 'timezone'], axis=1, inplace = True)

In [19]:
# Split date and time in the dataframe
df2['Date'],df2['Time'] = zip(*[(d.date(), d.time()) for d in df2['Reading_date']])

In [20]:
# drop reading_date after the split for a clean and standard DataFrame
df2.drop(['Reading_date'], axis=1, inplace = True)

In [21]:
# first step: install and create a folder
# as account identifier to secure your snowflake login details(.env)

#!pip install python-dotenv
#!pip install snowflake-sqlalchemy
#!pip install snowflake-connector-python


### Loading To database

In [22]:
engine = create_engine(URL(
        user = os.getenv("DB_USER"),
        password = os.getenv("DB_PASSWORD"),
        account = os.getenv("ACCOUNT_IDENTIFIER"),
        database = os.getenv("DB_NAME"),
        schema = os.getenv("DB_SCHEMA"),
        warehouse = os.getenv("DB_WAREHOUSE")
))

In [23]:
load_dotenv(override=True)
os.getenv('password')

In [ ]:
df2.to_sql('WEATHER', con=engine, if_exists='append',index=False)

In [ ]:
#print(len(os.environ['password']))

In [ ]:
#print(os.environ['schema'])

For_Staging
